# Dataset on Atlas/Hive

<a href="https://colab.research.google.com/github/VertaAI/modeldb-client/blob/master/workflows/demos/sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Cell use ful for live development

In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# restart your notebook if prompted on Colab
try:
    import verta
except ModuleNotFoundError:
    !pip3 install verta

In [ ]:
try:
    from pyhive import hive
except ModuleNotFoundError:
    !pip3 install pyhive
    !pip3 install thrift
    !pip3 install sasl
    !pip3 install thrift_sasl
    from pyhive import hive

In [ ]:
HOST = "demo.app.verta.ai"

PROJECT_NAME = "New york Cab demand Prediction- S3 demo CC"
EXPERIMENT_NAME = "Experiment Only to showcase Datasets"

In [ ]:
#import os
#os.environ['VERTA_EMAIL'] = ''
#os.environ['VERTA_DEV_KEY'] = ''

## Imports

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

import itertools
from multiprocessing import Pool
import os
import time

import six

import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics

---

# Read connection information for Atlas/Hive from environment

In [ ]:
atlas_user_name = ""
atlas_password = ""
hive_password = ""
atlas_url = %env ATLAS_URL
atlas_user_name = %env ATLAS_USER_NAME
atlas_password = %env ATLAS_PASSWORD
hive_url =  %env HIVE_URL
hive_password = %env HIVE_PASSWORD
print ("Atlas username set : " + str(atlas_user_name!=""))
print ("Atlas password set : " + str(atlas_password!=""))
print ("Hive password set : " + str(hive_password!=""))
[atlas_url, hive_url]

# Log Workflow

## Instantiate Client

In [ ]:
from verta import Client
from verta.utils import ModelAPI

client = Client(HOST)
proj = client.set_project(PROJECT_NAME)
expt = client.set_experiment(EXPERIMENT_NAME)

## Create Dataset and Dataset version

In [ ]:
dataset = client.create_atlas_hive_dataset("NYC Cab Dataset From Atlas + Hive")

In [ ]:
atlas_entity_endpoint = "/api/atlas/v2/entity/bulk"
atlas_guid = "d2fdde40-706f-44af-afde-155177b8d2e4"

In [ ]:
version =  client.create_atlas_hive_dataset_version(dataset, atlas_guid, 
                                                   atlas_url, atlas_user_name,
                                                   atlas_password)

---

# Fetcing data from Hive

In [ ]:
table_name = list(filter(lambda x: x.key=="table_name", version.attrs))[0].value.string_value
database_name = list(filter(lambda x: x.key=="database_name", version.attrs))[0].value.string_value
query = "select * from "+database_name+"."+table_name

In [ ]:
cursor = hive.connect(hive_url).cursor()
cursor.execute(query)

data = cursor.fetchall()

col_names =  [ x[0] for x in cursor.description]
data_frame = pd.DataFrame( data, columns=col_names)
data_frame.head()